Les bibliotheques

In [ ]:
from config.config import PROCESSED_DATA_DIR, ANNOTATIONS_DIR, CLASSES_FILE
from transformers import AutoProcessor, AutoModelForCausalLM  
from PIL import Image
import requests
import copy
import torch
from tqdm import tqdm
import os
import gc
%matplotlib inline  

2025-06-23 13:23:33.279306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750685013.477213      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750685013.535683      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device used here is: {device}")

device used here is: cuda


In [6]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype='auto').eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


...

processing_florence2.py:   0%|          | 0.00/48.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Les fichiers

In [ ]:
input_folder = PROCESSED_DATA_DIR
output_folder = ANNOTATIONS_DIR
os.makedirs(output_dir, exist_ok=True)

In [9]:
def load_class_map(classes_file):
    custom_class_map = {}
    with open(classes_file, "r") as f:
        for idx, line in enumerate(f):
            label = line.strip()
            custom_class_map[label] = idx
    return custom_class_map

Liste des classes COCO

In [2]:
custom_class_map = {}
class_counter = 0

In [ ]:
classes_file_path = CLASSES_FILE
custom_class_map = load_class_map(classes_file_path)
class_counter = max(custom_class_map.values()) + 1  

In [3]:
def run_example(task_prompt, image):
    inputs = processor(text=task_prompt, images=image, return_tensors="pt").to("cuda", torch.float16)

    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        early_stopping=False,
        do_sample=False,
        num_beams=1  # Réduit la mémoire GPU utilisée
    )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )
    return parsed_answer


Conversion des résultats Florence -> YOLO

In [11]:
def convert_bboxes_to_yolo(bboxes, labels, image_width, image_height):
    global custom_class_map, class_counter
    yolo_lines = []
    for box, label in zip(bboxes, labels):
        x1, y1, x2, y2 = box
        x_center = (x1 + x2) / 2 / image_width
        y_center = (y1 + y2) / 2 / image_height
        width = (x2 - x1) / image_width
        height = (y2 - y1) / image_height

        if label not in custom_class_map:
            custom_class_map[label] = class_counter
            print(f"Nouvelle classe détectée : {label} → ID {class_counter}")
            class_counter += 1

        class_id = custom_class_map[label]
        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    return yolo_lines

Nettoyage du résultat Florence

In [12]:
def convert_to_od_format(data):
    return {
        'bboxes': data.get('<OD>', {}).get('bboxes', []),
        'labels': data.get('<OD>', {}).get('labels', [])
    }

Pipeline complet multi-image

In [ ]:
def process_all_images(input_dir, output_dir):
    images = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif'))]
    task_prompt = "<OD>"

    for img_file in tqdm(images, desc="Traitement des images"):
        try:
            image_path = os.path.join(input_dir, img_file)
            image = Image.open(image_path).convert("RGB")

            parsed = run_example(task_prompt, image)
            result = convert_to_od_format(parsed)

            yolo_lines = convert_bboxes_to_yolo(result['bboxes'], result['labels'], image.width, image.height)

            # Sauvegarde des annotations
            txt_file = os.path.splitext(img_file)[0] + ".txt"
            with open(os.path.join(output_dir, txt_file), "w") as f:
                f.write("\n".join(yolo_lines))

        except Exception as e:
            print(f"Erreur avec {img_file} : {e}")

        finally:
            # Libérer la mémoire GPU après chaque image
            del image
            torch.cuda.empty_cache()
            gc.collect()

    # === 8. Génération du fichier des classes
    classes_output_path = os.path.join(output_dir, "classes.txt")
    if not os.path.exists(classes_output_path):
        with open(classes_output_path, "w") as f:
            for label, idx in sorted(custom_class_map.items(), key=lambda x: x[1]):
                f.write(f"{label}\n")

In [14]:
process_all_images(input_folder, output_folder)

Traitement des images:   1%|          | 6/956 [00:06<13:10,  1.20it/s]

Nouvelle classe détectée : plate → ID 76


Traitement des images:   2%|▏         | 17/956 [00:14<11:50,  1.32it/s]

Nouvelle classe détectée : toilet → ID 77


Traitement des images:   3%|▎         | 24/956 [00:19<11:50,  1.31it/s]

...

Traitement des images:  80%|████████  | 767/956 [09:55<02:25,  1.30it/s]

Nouvelle classe détectée : blanket → ID 105


Traitement des images:  85%|████████▍ | 808/956 [10:26<01:53,  1.31it/s]

Nouvelle classe détectée : sandal → ID 106


Traitement des images: 100%|██████████| 956/956 [12:21<00:00,  1.29it/s]


In [21]:
custom_class_map

{'bird': 0,
 'flagpole': 1,
 'suitcase': 2,
 'car': 3,
 'ball': 4,
 'bench': 5,
 'nail': 6,
 'mobile phone': 7,
 'bathtub': 8,
 'kite': 9,
 'door': 10,
 'frisbee': 11,
 'flowerpot': 12,
 'airplane': 13,
 'surfboard': 14,
 'hinge': 15,
 'window': 16,
 'umbrella': 17,
 'painting': 18,
 'flower': 19,
 'curtain': 20,
 'street light': 21,
 'houseplant': 22,
 'bed': 23,
 'door handle': 24,
 'plastic bag': 25,
 'book': 26,
 'pay phone': 27,
 'stairs': 28,
 'crossbar': 29,
 'towel': 30,
 'cattle': 31,
 'person': 32,
 'sheep': 33,
 'log': 34,
 'coin': 35,
 'mushroom': 36,
 'christmas tree': 37,
 'footwear': 38,
 'animal': 39,
 'power plugs and sockets': 40,
 'tarp': 41,
 'ladder': 42,
 'vent': 43,
 'flag': 44,
 'envelope': 45,
 'window blind': 46,
 'toilet paper': 47,
 'stop sign': 48,
 'adhesive tape': 49,
 'tennis racket': 50,
 'insect': 51,
 'trash bin/can': 52,
 'fish': 53,
 'clock': 54,
 'backpack': 55,
 'toothbrush': 56,
 'banner': 57,
 'house': 58,
 'refrigerator': 59,
 'box': 60,
 'hook